# Simulate genotype data

Following the work of [P. Casale](https://www.ebi.ac.uk/sites/ebi.ac.uk/files/shared/documents/phdtheses/Casale-Thesis.pdf), we simulated cohorts with different sample sizes based on genotype data from the 1000 Genomes Project (Phase 3).

1. for each newly synthesised individual we randomly select A ancestors
2. the new genotype is built as a mosaic of blocks of 1,000 variants, where each block is copied from the corresponding block in one of the A ancestors (selected at random)

## Genotype data for runtime experiments

For runtime experiments, we generated cohorts with 500, 1K, 2K, 5K, 10K, 20K, 50K, 100K, 200K and 500K individuals. We first assigned each individual to one of the 26 populations in the 1000 Genomes Project and then we sampled A = 10 ancestors from that population, generating cohorts of unrelated individuals and preserving the population structure in the original data.

## Genotype data for calibration experiments

For calibration experiments, we generated three cohorts with 1,000 individuals and different genetic structures, considering only European populations (CEU, FIN, GBR, IBS and TSI).

- Population Structure (*simPopStructure*). 
    We simulated population structure by (i) assigning each newly synthesised individual to one of the five European populations (CEU, FIN, GBR, IBS and TSI) and (ii) considering A = 10 ancestors from that population.

- Unrelated individuals (*simUnrelated*).  
    For each synthesised individual, we considered A = 10 ancestors, which were randomly sampled from the whole set of Europeans.

- Related Individuals (*simRelated*). 
    For each synthesised individual, we considered A = 2 ancestors, again sampled from the whole set of Europeans.

In [3]:
import pandas as pd
from random import randint, seed
import gzip

In [9]:
# Read metadata
df = pd.read_csv('../data/metadata.tsv', sep = '\t')
df = pd.DataFrame(df)

# Remove NaN columns
df = df.dropna(axis = 1)

df.head()

,sample,pop,super_pop,gender
0,HG00096,GBR,EUR,male
1,HG00097,GBR,EUR,female
2,HG00099,GBR,EUR,female
3,HG00100,GBR,EUR,female
4,HG00101,GBR,EUR,male


### *simUnrelated*

In [31]:
# Define simulation parameters
A = 10
n = 10000
blocksize = 1000
eur = True
popstr = True

ancestors = dict()

seed(123)  
if eur:
    if popstr:
        pops = list(df.loc[df['super_pop']=='EUR', 'pop'].unique())
        for i in range(n):
            pop = pops[randint(0, len(pops)-1)]
            inds = list(df.loc[df['pop'] == pop, 'sample'])
            ancestors[i] = [inds[randint(0, len(inds)-1)] for i in range(A)]
            break
    else:
        inds = list(df.loc[df['super_pop'] == 'EUR', 'sample'])
        for i in range(n):
            ancestors[i] = [inds[randint(0, len(inds)-1)] for i in range(A)]
else:
    if popstr:
        pops = list(df['pop'].unique())
        for i in range(n):
            pop = pops[randint(0, len(pops)-1)]
            inds = list(df.loc[df['pop'] == pop, 'sample'])
            ancestors[i] = [inds[randint(0, len(inds)-1)] for i in range(A)]
    else:
        inds = list(df['sample'])
        for i in range(n):
            ancestors[i] = [inds[randint(0, len(inds)-1)] for i in range(A)]
            

GBR
['HG00096', 'HG00097', 'HG00099', 'HG00100', 'HG00101', 'HG00102', 'HG00103', 'HG00105', 'HG00106', 'HG00107', 'HG00108', 'HG00109', 'HG00110', 'HG00111', 'HG00112', 'HG00113', 'HG00114', 'HG00115', 'HG00116', 'HG00117', 'HG00118', 'HG00119', 'HG00120', 'HG00121', 'HG00122', 'HG00123', 'HG00125', 'HG00126', 'HG00127', 'HG00128', 'HG00129', 'HG00130', 'HG00131', 'HG00132', 'HG00133', 'HG00136', 'HG00137', 'HG00138', 'HG00139', 'HG00140', 'HG00141', 'HG00142', 'HG00143', 'HG00145', 'HG00146', 'HG00148', 'HG00149', 'HG00150', 'HG00151', 'HG00154', 'HG00155', 'HG00157', 'HG00158', 'HG00159', 'HG00160', 'HG00231', 'HG00232', 'HG00233', 'HG00234', 'HG00235', 'HG00236', 'HG00237', 'HG00238', 'HG00239', 'HG00240', 'HG00242', 'HG00243', 'HG00244', 'HG00245', 'HG00246', 'HG00250', 'HG00251', 'HG00252', 'HG00253', 'HG00254', 'HG00255', 'HG00256', 'HG00257', 'HG00258', 'HG00259', 'HG00260', 'HG00261', 'HG00262', 'HG00263', 'HG00264', 'HG00265', 'HG01334', 'HG01789', 'HG01790', 'HG01791', 'HG02

In [4]:
def formatGT(genotypes):
    for i, gt in enumerate(genotypes):
        if (gt == '0|0'):
            genotypes[i] = "0"
        elif(gt == '1|0' or genotypes[i] == '0|1'):
            genotypes[i] = "1"
        elif(gt == '1|1'):
            genotypes[i] = "2"
    return(genotypes)
    

In [19]:
%%timeit -n 1 -r 1
import cProfile, pstats
pr = cProfile.Profile()
pr.enable()
      
fh = gzip.open('../data/genotypes.vcf.gz', 'rt')
for line in fh:
        if line.startswith('##'):
            # skip comment lines
            continue
        elif line.startswith('#CHROM'):
            # read header
            header = line.strip().split("\t")
            count = 0
            # print ('\t'.join(header[0:9]) + '\t' + '\t'.join(['sim' + str(i) for i in range(n)]), '\n')
        else:
            # For every block variants, change ancestors
            if (count % blocksize == 0):
                ancestors_block_idx = [header.index(ancestors[i][randint(0, A-1)]) for i in range(n)]
            line = line.strip().split('\t')
            gt = [line[idx] for idx in ancestors_block_idx]
            # Obtain genotypes corresponding to the selected ancestors
            # print('\t'.join(line[0:9]) + '\t' + '\t'.join(gt) + '\n')
            # print('\t'.join(gt))
            count += 1
            if(count == 10000): break

pr.disable()
# pr.print_stats()

         487310 function calls in 2.173 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     1000    0.012    0.000    0.058    0.000 <magic-timeit>:16(<listcomp>)
       10    0.009    0.001    0.235    0.023 <magic-timeit>:22(<listcomp>)
    10000    0.477    0.000    0.477    0.000 <magic-timeit>:24(<listcomp>)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:23(getpreferredencoding)
    12483    0.007    0.000    0.009    0.000 _compression.py:12(_check_not_closed)
        1    0.000    0.000    0.000    0.000 _compression.py:150(tell)
        1    0.000    0.000    0.000    0.000 _compression.py:36(readable)
        1    0.000    0.000    0.000    0.000 _compression.py:39(__init__)
    12483    0.060    0.000    0.572    0.000 _compression.py:66(readinto)
        1    0.000    0.000    0.000    0.000 codecs.py:259(__init__)
        1    0.000    0.000    0.000    0.000 codecs.py:308(__init__)
    12483    0

NameError: name 'pr' is not defined